In [1]:
!wget https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar -O /root/.pyterrier/custom-terrier-token-processing-0.0.1.jar

--2024-06-27 11:46:53--  https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499865236 (477M) [application/java-archive]
Saving to: ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’

/root/.pyterrier/cu 100%[===================>] 476.71M  96.6MB/s    in 5.5s    

2024-06-27 11:46:59 (87.1 MB/s) - ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’ saved [499865236/499865236]



In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
#from jnius import autoclass

ensure_pyterrier_is_loaded()
tira = Client()

pd.set_option('display.max_colwidth', 0)
#'mam10eks:custom-terrier-token-processing:0.0.1'
#'com.github.terrierteam:terrier-prf:-SNAPSHOT'
if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
stopwords= ["I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
              "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
              "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
              "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
              "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
                "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
                  "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
                    "at", "by", "for", "with", "about", "against", "between", "into", "through", 
                    "during", "before", "after", "above", "below", "to", "from", "up", "down", "in",
                      "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
                      "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
                        "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", 
                        "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [4]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [5]:
stemmer = pt.TerrierStemmer.porter
stemmer1= pt.TerrierStemmer.weakporter
tokenizer = pt.TerrierTokeniser.english

#lemmatizer = autoclass("org.terrier.terms.StanfordLemmatizer")()
#stemmer2 = autoclass("org.terrier.terms.LemurKrovetzStemmer")

iter_indexer = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, tokeniser=tokenizer, stemmer =stemmer, meta={'docno': 100, 'text': 4096})
#iter_indexer = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stopwords =None, stemmer =None, meta={'docno': 100, 'text': 4096})

print('start')
indexref = iter_indexer.index(tqdm(pt_dataset.get_corpus_iter()))
print('end')

start


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90006/126958 [00:26<00:05, 6575.16it/s]

11:47:36.039 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:32<00:00, 3897.61it/s] 
100%|██████████| 126958/126958 [00:32<00:00, 3897.77it/s]


11:47:46.717 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents
end


In [6]:
tf_idf = pt.BatchRetrieve(indexref, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")
dph = pt.BatchRetrieve(indexref, wmodel="DPH")
linear = 0.5*bm25+tf_idf + 2* dph

In [7]:

bm252 = pt.BatchRetrieve(indexref, wmodel="BM25") >> pt.pipelines.PerQueryMaxMinScoreTransformer()
dph2 = pt.BatchRetrieve(indexref, wmodel="DPH") >> pt.pipelines.PerQueryMaxMinScoreTransformer()
tf_idf2= pt.BatchRetrieve(indexref, wmodel="TF_IDF") >> pt.pipelines.PerQueryMaxMinScoreTransformer()
linear2 = 0.25 * tf_idf2 + 0.25*bm252 + 0.5 * dph2

In [8]:
#pt.rewrite.RM3(indexref)
#DPH qe best so far
dph_qe = pt.BatchRetrieve(indexref, wmodel="DPH", controls={"qe":"on", "qemodel" : "KLCorrect"},)
dph_qe2 = pt.BatchRetrieve(indexref, wmodel="DPH", controls={"qe":"on", "qemodel" : "Bo1"},)
linear3 = 0.5*dph_qe + dph
#mdl = pt.BatchRetrieve(indexref, wmodel="PL2")


In [9]:
pipe = (pt.BatchRetrieve(indexref, wmodel="DPH", controls={"qe":"on", "qemodel" : "KLCorrect"}) >> 
    pt.rewrite.RM3(indexref) >> 
    pt.BatchRetrieve(indexref, wmodel="DPH", controls={"qe":"on", "qemodel" : "KLCorrect"})
)

In [10]:
pt_dataset.get_topics('text').head(3)

print('Now we do the retrieval...')
run = dph_qe(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

persist_and_normalize_run(run, system_name='altered index 1', default_output='../runs/anotherExperiment')

Now we do the retrieval...
Done. Here are the first 10 entries of the run
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/anotherExperiment".
Done. run file is stored under "../runs/anotherExperiment/run.txt".
